In [1]:
import pandas as pd
import datetime

In [2]:
pwd

u'/Users/aurelperianu/Documents/Medicare_payments_classifier'

In [3]:
#data_all=pd.read_csv('Medicare_Data/2012MedicareProviderPayment.txt', delimiter='\t')
provider_all=pd.read_csv('Medicare_Data/Physician_Compare_2014/National_Downloadable_File.csv')

/Users/aurelperianu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,25,26,28,29,30,31,32,33,34,36,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#fct rename columns lower letters; remove spaces
def lower_columns(pdx):
    cols = pdx.columns.tolist()
    cols = [col.lower() for col in cols]
    pdx.columns = cols
    pdx.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
    return pdx
provider_all=lower_columns(provider_all)

In [5]:
#add zip code
provider_all['zip_code']=provider_all['zip_code'].astype(str)
provider_all['zip_base']=provider_all['zip_code'].str.slice(0,5)

#data_all['nppes_provider_zip_base'].unique()

In [6]:
#provider_all['professional_accepts_medicare_assignment']=provider_all['professional_accepts_medicare_assignment'].astype(str)
#provider_all['state']=provider_all['state'].astype(str)

In [7]:
#select WA state providers
#provider_wa['professional_accepts_medicare_assignment'=='M']
#provider_wa=provider_all[provider_all['state']=='WA']
#provider_wa.info()

In [8]:
#provider_all['professional_accepts_medicare_assignment'].unique()

In [9]:
provider_all.head().T

,0,1,2,3,4
npi,1821199704,1487927612,1235146762,1285727842,1770581647
pac_id,9739343922,4880850486,2365435336,5799762829,6608976915
professional_enrollment_id,I20120620000327,I20120726000331,I20040406000367,I20040707000479,I20070714000216
last_name,SMITH,HALL,WHITE,CASHA,FACCA
first_name,WILLIAM,ESTHER,BARBARA,MARK,ROBERT
middle_name,A,S,L,S,R
suffix,NaN,NaN,NaN,NaN,NaN
gender,M,F,F,M,M
credential,NaN,NaN,CSW,DC,NaN
medical_school_name,PALMER COLLEGE CHIROPRACTIC - WEST SUNNYVALE,LIFE CHIROPRACTIC COLLEGE - WEST,OTHER,OTHER,LIFE CHIROPRACTIC COLLEGE - WEST


In [10]:
#drop columns
drop_col=['professional_accepts_medicare_assignment','marker_of_address_line_2_suppression',\
         'participated_in_the_medicare_maintenance_of_certification_program.']
def drop_columns(col_remove,pdx):
    columns=list(pdx.columns.values)
    for col in col_remove:
        if col in columns:
            pdx.drop(col, axis=1, inplace=True)
    return pdx

In [11]:
provider_all=drop_columns(drop_col, provider_all)

In [12]:
provider_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247118 entries, 0 to 2247117
Data columns (total 40 columns):
npi                                                                  int64
pac_id                                                               int64
professional_enrollment_id                                           object
last_name                                                            object
first_name                                                           object
middle_name                                                          object
suffix                                                               object
gender                                                               object
credential                                                           object
medical_school_name                                                  object
graduation_year                                                      float64
primary_specialty                                   

In [13]:
#change column names
#col_to_rename={'participated_in_the_medicare_maintenance_of_certification_program.':'medicare_certified'\
#               ,'committed_to_heart_health_through_the_million_hearts®_initiative.':'hearth_initiative'}
#provider_all.rename(columns=col_to_rename, inplace=True)

In [14]:
#rename columns
#fct remane columns
def name_columns(value_old,value_new,pdx):
    cols = pdx.columns.tolist()
    cols = [col.replace(value_old,value_new) for col in cols]
    pdx.columns = cols
    return pdx
provider_all=name_columns('professional_enrollment_id', 'prof_id', provider_all)
provider_all=name_columns('hospital_affiliation', 'related_org', provider_all)
provider_all=name_columns('secondary_specialty', 'specialty', provider_all)
#provider_all=name_columns('participated_in_the_medicare_maintenance_of_certification_program.', 'medicare_certified', provider_all)
provider_all=name_columns('committed_to_heart_health_through_the_million_hearts®_initiative.', 'hearth_initiative', provider_all)


In [15]:
provider_all=name_columns('number_of_group_practice_members', 'practice_size', provider_all)
provider_all=name_columns('all_secondary_specialties', 'secondary_spec', provider_all)
provider_all=name_columns('reported_quality_measures', 'quality_measures', provider_all)
provider_all=name_columns('used_electronic_health_records', 'electronic_measures', provider_all)
provider_all=name_columns('organization_legal_name', 'org_name', provider_all)
provider_all=name_columns('line_1_street_address', 'address', provider_all)
provider_all=name_columns('line_2_street_address', 'address_extra', provider_all)
provider_all=name_columns('group_practice_pac_id', 'group_pac_id', provider_all)
provider_all=name_columns('medical_school_name', 'medical_school', provider_all)


In [16]:
#str_cols = provider_all.columns[provider_all.dtypes==object]
#provider_all[str_cols] = provider_all[str_cols].fillna('.')


#provider_all.fillna(0,inplace=True)

In [17]:
#convert columns to numeric
#provider_all=provider_all.apply(lambda x: pd.to_numeric(x, errors='ignore'))

numeric=['npi','pac_id','graduation_year','group_pac_id','practice_size','related_org_ccn_1','related_org_ccn_2','related_org_ccn_3',\
        'related_org_ccn_4','related_org_ccn_5','zip_base']

provider_all[numeric] = provider_all[numeric].apply(pd.to_numeric, errors='coerce')

#pd.to_numeric(s, errors='coerce')

In [18]:
#fill all NAN
provider_all=provider_all.replace(['nan','NaN'], ['.','0'])
provider_all.fillna(0,inplace=True)

#convert columns to string
# non_numeric=['prof_id','last_name','first_name','middle_name','suffix','gender','credential','medical_school','primary_specialty',\
#              'specialty_1','specialty_2','specialty_3','specialty_4','secondary_spec','org_name','address','address_extra','city',\
#              'state','zip_code','phone_number','related_org_lbn_1','related_org_lbn_2','related_org_lbn_3','related_org_lbn_4',\
#              'related_org_lbn_5','quality_measures','electronic_measures','hearth_initiative']
non_numeric= provider_all.columns[provider_all.dtypes==object]
provider_all[non_numeric] = provider_all[non_numeric].astype(str)
provider_all['group_pac_id'] = provider_all['group_pac_id'].astype(int)
#provider_all=provider_all.replace('nan', '.')

In [19]:
#provider_all.info()

In [20]:
#provider_all.head(200).T

In [21]:
#set an index
provider_all=provider_all.set_index(provider_all.npi)

In [22]:
#graduation year-convert to years of experience

def f(row):
    now = datetime.datetime.now()
    curr_year=now.year
    if row == 0:
        val = 0
    else: 
        val=curr_year-row
    return val
provider_all['years_experience'] = provider_all['graduation_year'].apply(f)


In [23]:
org_cols=['prof_id',
 'org_name',
 'group_pac_id',
 'practice_size',
 'address',
 'address_extra',
 'city',
 'state',
 'zip_code',
 'phone_number',
 'related_org_ccn_1',
 'related_org_lbn_1',
 'related_org_ccn_2',
 'related_org_lbn_2',
 'related_org_ccn_3',
 'related_org_lbn_3',
 'related_org_ccn_4',
 'related_org_lbn_4',
 'related_org_ccn_5',
 'related_org_lbn_5',
 'zip_base']
#drop group columns
org_all=provider_all[org_cols]

#provider_all.info()

# second columns drop for group
org_cols2=[
 'prof_id',
 'address',
 'address_extra',
 'city',
 'zip_code',
 'phone_number',
 'related_org_ccn_1',
 'related_org_lbn_1',
 'related_org_ccn_2',
 'related_org_lbn_2',
 'related_org_ccn_3',
 'related_org_lbn_3',
 'related_org_ccn_4',
 'related_org_lbn_4',
 'related_org_ccn_5',
 'related_org_lbn_5',
 'zip_base']

org_all=drop_columns(org_cols2, org_all)



/Users/aurelperianu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
print org_cols

['prof_id', 'org_name', 'group_pac_id', 'practice_size', 'address', 'address_extra', 'city', 'state', 'zip_code', 'phone_number', 'related_org_ccn_1', 'related_org_lbn_1', 'related_org_ccn_2', 'related_org_lbn_2', 'related_org_ccn_3', 'related_org_lbn_3', 'related_org_ccn_4', 'related_org_lbn_4', 'related_org_ccn_5', 'related_org_lbn_5', 'zip_base']


In [25]:
provider_drop = [x for x in org_cols if x != 'state']
# provider_drop=org_cols.remove('state')
#print provider_drop
provider_all=drop_columns(provider_drop, provider_all)
specialty_cols=[
    'specialty_1',
    'specialty_2',
    'specialty_3',
    'specialty_4']
provider_all=drop_columns(specialty_cols, provider_all)

In [26]:
org_all.shape

(2247118, 4)

In [27]:
provider_all.shape

(2247118, 17)

In [28]:
#remove duplicates for provider and group files
org_all=org_all.drop_duplicates()
provider_all=provider_all.drop_duplicates()
#org_all['org_name'].value_counts()

In [29]:
org_all.shape

(183975, 4)

In [30]:
provider_all.shape

(1060454, 17)

In [31]:
#DO NOT DELETE - REFERENCE DUPLICATES SPECIALITIES
#provider_all[provider_all['npi']==1073536074].head().T

In [46]:
#select WA data
wa_provider=provider_all[provider_all['state']=='WA']
wa_org=org_all[org_all['state']=='WA']

In [47]:
#save WA data
wa_provider.to_pickle('wa_provider_pickle.pkl') 
wa_org.to_pickle('wa_org_pickle.pkl') 

In [48]:
#load from pickle
#wa_provider= pd.read_pickle('wa_provider_pickle.pkl')
#wa_org= pd.read_pickle('wa_org_pickle.pkl')

In [49]:
wa_provider.head()

,npi,pac_id,last_name,first_name,middle_name,suffix,gender,credential,medical_school,graduation_year,primary_specialty,secondary_spec,state,quality_measures,electronic_measures,hearth_initiative,years_experience
npi,,,,,,,,,,,,,,,,,
1023030962,1023030962,345315404,BASTRON,MICHAEL,K,0,M,0,PALMER COLLEGE CHIROPRACTIC - WEST SUNNYVALE,1990.0,CHIROPRACTIC,0,WA,0,0,0,27.0
1720088685,1720088685,8921040635,LITTMAN,THOMAS,A,0,M,AU,OTHER,1990.0,AUDIOLOGIST,0,WA,0,0,0,27.0
1295989689,1295989689,2466513460,JOHNSON,ERIKA,L,0,F,0,OTHER,2008.0,PHYSICAL THERAPY,0,WA,0,0,0,9.0
1922016203,1922016203,648178822,ULLMAN,RONALD,H,0,M,MD,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1967.0,OPHTHALMOLOGY,0,WA,0,0,0,50.0
1841608809,1841608809,244540201,MAHER,ANITA,D,0,F,0,OTHER,2014.0,OPTOMETRY,0,WA,0,0,0,3.0


In [37]:
#org_all = org_all.sort(org_all.columns[1],ascending=False)
#org_all.head(10)

In [38]:
#Build summary variables - in-work

# secondary_spec
# import pandas as pd

# def alert(c):
#   if c['used'] == 1.0:
#     return 'Full'
#   elif c['used'] == 0.0:
#     return 'Empty'
#   elif 0.0 < c['used'] < 1.0:
#     return 'Partial'
#   else:
#     return 'Undefined'

# df = pd.DataFrame(data={'portion':[1, 2, 3, 4], 'used':[1.0, 0.3, 0.0, 0.8]})

# df['alert'] = df.apply(alert, axis=1)

######combine specialities
# def split_spec(df):
#     var=set()
#     words=df['secondary_spec'].split(',')
#     var.update(words)
#     var.add(df['primary_specialty'])
#     var.add(df['specialty_1'])
#     var.add(df['specialty_2'])
#     var.add(df['specialty_3'])
#     #var.add(df['specialty_4'])
#     return var
# provider_all['years_experience'] = provider_all.apply(f, axis=1)



In [39]:
#provider_all.duplicated().value_counts()
#provider_all=provider_all.drop_duplicates()
#provider_all.drop_duplicates(*args, **kwargs)
#provider_all['npi'].value_counts()

In [40]:
#provider_wa=provider_all[provider_all['state']=='WA']
#provider_wa.info()

In [41]:
#fill all NaN with 0
#provider_wa.update(provider_wa.fillna(0))

In [42]:
#transform secondary speciality in counts
#transform hospital_affiliation in counts
#change col names

In [43]:
#provider_wa=provider_all[provider_all['state']=='WA']
#provider_wa.info()

In [44]:
#provider_wa['professional_accepts_medicare_assignment'=='M']
#x=provider_wa[provider_wa['professional_accepts_medicare_assignment']=='M']

In [45]:
#provider_wa['hospital_affiliation_lbn_5'].value_counts()